# CNN from scratch

## Setup


In [9]:
import torch
import torch.nn as nn
from tensorboardX import SummaryWriter
from pa2_sample_code import get_datasets

train_data, eval_data = get_datasets()

# split train set into holdout_train and holdout_eval sets
holdout_train_len = int(len(train_data) * 0.8)
holdout_eval_len = len(train_data) - holdout_train_len
holdout_train_data, holdout_eval_data = torch.utils.data.random_split(train_data, [holdout_train_len, holdout_eval_len])

## Declare model

In [32]:
class CnnClassifier(nn.Module):
    def __init__(self, n_hidden):
        super(CnnClassifier, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=4, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels=4, out_channels=8, kernel_size=3, stride=2, padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=2, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=1, padding=0),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=0),
            nn.Sigmoid()
        )
        
        self.predictor = nn.Sequential(
            nn.Linear(in_features=32, out_features=n_hidden),
            nn.ReLU(),
            nn.Linear(in_features=n_hidden, out_features=47)
        )
        
        self.loss_func = nn.CrossEntropyLoss(reduction='sum')
        
    def forward(self, in_data):
        img_features = self.encoder(in_data).view(in_data.size(0), 32)
        logits = self.predictor(img_features)
        return logits

    def loss(self, logits, labels):
        return self.loss_func(logits, labels) / logits.size(0)
    
    def top_k_acc(self, logits, labels, k=1):
        _, k_labels_pred = torch.topk(logits, k=k, dim=1) # shape (n, k)
        k_labels = labels.unsqueeze(dim=1).expand(-1, k) # broadcast from (n) to (n, 1) to (n, k)
        # flatten tensors for comparison
        k_labels_pred_flat = k_labels_pred.reshape(1,-1).squeeze()
        k_labels_flat = k_labels.reshape(1,-1).squeeze()
        # get num_correct
        num_correct = k_labels_pred_flat.eq(k_labels_flat).sum(0).float().item() # get float for calculation
        return num_correct / labels.size(0)
        

## Training

In [20]:
def train(model, loaders, optimizer, writer, num_epoch=10, device='cpu'):
    # for logging to tensorboard
    loss = {}
    top1 = {}
    top3 = {}
    def run_epoch(mode):
        epoch_loss = 0.0
        epoch_top1 = 0.0
        epoch_top3 = 0.0
        for i, batch in enumerate(loaders[mode], 0):
            in_data, labels = batch
            in_data, labels = in_data.to(device), labels.to(device)

            if mode == 'train':
                optimizer.zero_grad()

            logits = model(in_data)
            batch_loss = model.loss(logits, labels)
            batch_top1 = model.top_k_acc(logits, labels, k=1)
            batch_top3 = model.top_k_acc(logits, labels, k=3)

            epoch_loss += batch_loss.item()
            epoch_top1 += batch_top1
            epoch_top3 += batch_top3

            if mode == 'train':
                batch_loss.backward()
                optimizer.step()

        # sum of all batchs / num of batches
        epoch_loss /= i + 1 
        epoch_top1 /= i + 1
        epoch_top3 /= i + 1
        
        loss[mode] = epoch_loss
        top1[mode] = epoch_top1
        top3[mode] = epoch_top3
        
        print('epoch %d %s loss %.4f top1 %.4f top3 %.4f' % (epoch, mode, epoch_loss, epoch_top1, epoch_top3))
        # log to tensorboard
        if not (writer is None) and (mode == 'eval'):
            writer.add_scalars('%s_loss' % model.__class__.__name__,
                         tag_scalar_dict={'train': loss['train'], 
                                          'eval': loss['eval']}, 
                         global_step=epoch)
            writer.add_scalars('%s_top1' % model.__class__.__name__,
                         tag_scalar_dict={'train': top1['train'], 
                                          'eval': top1['eval']}, 
                         global_step=epoch)
            writer.add_scalars('%s_top3' % model.__class__.__name__,
                         tag_scalar_dict={'train': top3['train'], 
                                          'eval': top3['eval']}, 
                         global_step=epoch)
    for epoch in range(num_epoch):
        run_epoch('train')
        run_epoch('eval')

In [31]:
for n_hidden in [32, 64]:
    for optim_conf in [
        {'optim':'adam', 'lr':0.001},
        {'optim':'sgd', 'lr':0.1},
        {'optim':'sgd', 'lr':0.01}
    ]:
        model = CnnClassifier(n_hidden=n_hidden)
        if optim_conf['optim'] == 'adam':
            optimizer = torch.optim.Adam(model.parameters(), lr=optim_conf['lr'])
        else:
            optimizer = torch.optim.SGD(model.parameters(), lr=optim_conf['lr'])
        conf_str = str(n_hidden)+'_'+optim_conf['optim']+'_'+str(optim_conf['lr'])
        print(conf_str)
        train(
            model=model,
            loaders={
                'train': torch.utils.data.DataLoader(holdout_train_data, batch_size=64, shuffle=True),
                'eval': torch.utils.data.DataLoader(holdout_eval_data, batch_size=64, shuffle=True)
            },
            optimizer=optimizer, 
            writer=SummaryWriter('./logs/cnn_scratch/%s' % (conf_str)), 
            num_epoch=20, 
            device='cpu'
        )

32_adam_0.001
1.0
4.0
3.0
5.0
3.0
5.0
2.0
3.0
0.0
6.0
2.0
4.0
1.0
2.0
0.0
4.0
1.0
2.0
1.0
5.0
3.0
6.0
1.0
7.0
1.0
6.0
0.0
2.0
1.0
3.0
1.0
4.0
1.0
3.0
0.0
2.0
1.0
8.0
0.0
5.0
0.0
2.0
1.0
2.0
0.0
4.0
2.0
5.0
1.0
5.0
3.0
6.0
1.0
4.0
0.0
1.0
2.0
3.0
2.0
5.0
4.0
6.0
4.0
6.0
3.0
6.0
0.0
2.0
2.0
3.0
0.0
2.0
2.0
4.0
2.0
3.0
4.0
6.0
2.0
2.0
2.0
7.0
0.0
4.0
1.0
5.0
3.0
5.0
3.0
5.0
3.0
4.0
1.0
3.0
0.0
1.0
1.0
4.0
1.0
1.0
2.0
3.0
0.0
4.0
0.0
6.0
2.0
8.0
4.0
4.0
0.0
3.0
1.0
4.0
2.0
4.0
0.0
0.0
4.0
7.0
2.0
10.0
3.0
6.0
0.0
1.0
0.0
2.0
1.0
4.0
0.0
0.0
1.0
5.0
1.0
3.0
3.0
6.0
2.0
6.0
0.0
2.0
1.0
4.0
0.0
3.0
0.0
2.0
2.0
3.0
3.0
4.0
3.0
4.0
1.0
5.0
1.0
1.0
1.0
5.0
0.0
3.0
0.0
1.0
0.0
2.0
1.0
4.0
1.0
4.0
4.0
6.0
1.0
3.0
4.0
7.0
0.0
3.0
1.0
4.0
2.0
4.0
0.0
4.0
1.0
5.0
0.0
0.0
0.0
3.0
1.0
4.0
1.0
5.0
2.0
3.0
0.0
1.0
2.0
4.0
3.0
8.0
1.0
4.0
0.0
5.0
1.0
5.0
2.0
5.0
1.0
7.0
1.0
1.0
1.0
5.0
3.0
3.0
1.0
4.0
0.0
1.0
0.0
3.0
0.0
2.0
0.0
4.0
3.0
5.0
1.0
3.0
1.0
4.0
4.0
8.0
2.0
6.0
0.0
2.0
0.0
1.0
1.0
3.0
0.0
1.0
1

KeyboardInterrupt: 